In [5]:
import sys
sys.path.append('../')
from datetime import datetime
from utils.extract_amplitude_data import start_session
from utils.df_utils import fill_na_columns
actual_date = datetime.now().strftime("%Y-%m-%d")
from invest_per_state.utils.amplitude.api_call import get_rql_data
from invest_per_state.utils.utils import filter_by_date_range
import pandas as pd

## RQLs de Amplitude

In [6]:
#
# !: Se puede sacar el historial apartir de acá

In [7]:
data_rql_amplitude = get_rql_data(actual_date)
data_rql_amplitude

Iniciando solicitud...
Respuesta recibida en 7.21 segundos
Código de estado: 200
Procesando respuesta...
Datos guardados en '2025-04-14-amplitude_data.json'
Datos procesados correctamente
Tiempo total: 8.17 segundos


,publication_code,brand,state,Time,MX RQL Online (By LS)--All Users
0,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-03-15T00:00:00,4.0
1,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-03-16T00:00:00,5.0
2,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-03-17T00:00:00,7.0
3,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-03-18T00:00:00,14.0
4,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-03-19T00:00:00,18.0
...,...,...,...,...,...
99877,MX2454-benelli-vz-125i,Benelli,Guanajuato,2025-04-10T00:00:00,0.0
99878,MX2454-benelli-vz-125i,Benelli,Guanajuato,2025-04-11T00:00:00,0.0
99879,MX2454-benelli-vz-125i,Benelli,Guanajuato,2025-04-12T00:00:00,0.0
99880,MX2454-benelli-vz-125i,Benelli,Guanajuato,2025-04-13T00:00:00,0.0


Filtrar por las fechas de los RQLs a buscar

In [8]:
date_range = ["2025-03-31", "2025-04-01", "2025-04-02", "2025-04-03", "2025-04-04", "2025-04-05", "2025-04-06"]
data_rql_amplitude = filter_by_date_range(data_rql_amplitude, date_range)

Filtrar quitando RQLs vacíos

In [9]:
data_rql_amplitude.loc[:, "MX RQL Online (By LS)--All Users"] = data_rql_amplitude.loc[:, "MX RQL Online (By LS)--All Users"].astype(float)
data_rql_amplitude = data_rql_amplitude[data_rql_amplitude["MX RQL Online (By LS)--All Users"] != 0]
total_rqls = data_rql_amplitude['MX RQL Online (By LS)--All Users'].sum()
display(data_rql_amplitude)
print(f"Total de MX RQL Online (By LS)--All Users: {total_rqls}")

,publication_code,brand,state,Time,MX RQL Online (By LS)--All Users,day
16,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-03-31T00:00:00,15.0,2025-03-31
17,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-04-01T00:00:00,13.0,2025-04-01
18,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-04-02T00:00:00,24.0,2025-04-02
19,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-04-03T00:00:00,22.0,2025-04-03
20,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-04-04T00:00:00,20.0,2025-04-04
...,...,...,...,...,...,...
98909,MX2616-cf-moto-cforce-625,CF Moto,Guanajuato,2025-04-03T00:00:00,1.0,2025-04-03
99346,MX1223-honda-trx-250-tm-2023,Honda,México,2025-04-06T00:00:00,1.0,2025-04-06
99528,MX1202-kawasaki-vulcan-1700-vaquero-2023,Kawasaki,Puebla,2025-04-02T00:00:00,1.0,2025-04-02
99620,MX898-kymco-agility-200,Kymco,Quintana Roo,2025-04-01T00:00:00,1.0,2025-04-01


Total de MX RQL Online (By LS)--All Users: 4914.0


## Datos de Geolocalización

In [10]:
df_state_per_media = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\Geolocalización\MX equivalencias geo regiones - Equivalencias_Medios.csv")

Combinar datos de geolocalización

In [13]:
df_rql_per_state_and_brand_merged = pd.merge(data_rql_amplitude, df_state_per_media, left_on="state", right_on="Amplitude_ips", how = "left")
df_rql_per_state_and_brand_merged.drop(columns = ["Time", "publication_code", "Amplitude_ips"], inplace = True)
df_rql_per_state_and_brand_merged.rename(columns = {"MX RQL Online (By LS)--All Users": "rql"}, inplace = True)
df_rql_per_state_and_brand_merged = df_rql_per_state_and_brand_merged[["day", "state", "brand", "rql", "Meta", "GAds", "TikTok"]]
# df_rql_per_state_and_brand_merged.drop(columns = ["Amplitude_declarado"], inplace = True)

In [14]:
# Crear columnas de año y semana
df_rql_per_state_and_brand_merged['Año'] = pd.to_datetime(df_rql_per_state_and_brand_merged['day']).dt.year
df_rql_per_state_and_brand_merged['Semana'] = pd.to_datetime(df_rql_per_state_and_brand_merged['day']).dt.isocalendar().week
df_rql_per_state_and_brand_merged

,day,state,brand,rql,Meta,GAds,TikTok,Año,Semana
0,2025-03-31,México,Bajaj,15.0,State of Mexico,State of Mexico,Mexico,2025,14
1,2025-04-01,México,Bajaj,13.0,State of Mexico,State of Mexico,Mexico,2025,14
2,2025-04-02,México,Bajaj,24.0,State of Mexico,State of Mexico,Mexico,2025,14
3,2025-04-03,México,Bajaj,22.0,State of Mexico,State of Mexico,Mexico,2025,14
4,2025-04-04,México,Bajaj,20.0,State of Mexico,State of Mexico,Mexico,2025,14
...,...,...,...,...,...,...,...,...,...
3047,2025-04-03,Guanajuato,CF Moto,1.0,Guanajuato,Guanajuato,Guanajuato,2025,14
3048,2025-04-06,México,Honda,1.0,State of Mexico,State of Mexico,Mexico,2025,14
3049,2025-04-02,Puebla,Kawasaki,1.0,Puebla,Puebla,Puebla,2025,14
3050,2025-04-01,Quintana Roo,Kymco,1.0,Quintana Roo,Quintana Roo,Quintana Roo,2025,14


## RQL por cada una de las marcas

Se agrupan para evitar tener registros duplicados al hacer un combinado

In [22]:
# Agrupar por Marca, Semana y Año, y sumar el costo
rql_resumen_per_state_and_brand = df_rql_per_state_and_brand_merged.groupby(['day', 'state', 'brand', 'Meta', 'GAds', 'TikTok', 'Semana', 'Año'])['rql'].sum().reset_index()

In [23]:
rql_resumen_per_state_and_brand

,day,state,brand,Meta,GAds,TikTok,Semana,Año,rql
0,2025-03-31,Aguascalientes,Bajaj,Aguascalientes,Aguascalientes,Aguascalientes,14,2025,3.0
1,2025-03-31,Baja California,Bajaj,Baja California,Baja California,Baja California,14,2025,1.0
2,2025-03-31,Baja California,Dinamo,Baja California,Baja California,Baja California,14,2025,1.0
3,2025-03-31,Baja California,Honda,Baja California,Baja California,Baja California,14,2025,2.0
4,2025-03-31,Baja California,Vento,Baja California,Baja California,Baja California,14,2025,1.0
...,...,...,...,...,...,...,...,...,...
1083,2025-04-06,Yucatán,Royal Enfield,Yucatán,Yucatan,Yucatan,14,2025,1.0
1084,2025-04-06,Yucatán,Suzuki,Yucatán,Yucatan,Yucatan,14,2025,2.0
1085,2025-04-06,Yucatán,Yamaha,Yucatán,Yucatan,Yucatan,14,2025,2.0
1086,2025-04-06,Zacatecas,CF Moto,Zacatecas,Zacatecas,Zacatecas,14,2025,1.0


In [24]:
from datetime import datetime
actual_date = datetime.now().strftime("%Y%m%d")
print(actual_date)

20250414


In [26]:
rql_resumen_per_state_and_brand.to_csv(f"outputs/RQLs Amplitude/{actual_date}-rql_resumen_per_state_and_brand.csv", index=False)

## Verificación de cantidad de RQLs generados para una semana

In [13]:
resumen_rqls_por_marca_week = resumen_rqls_por_marca[resumen_rqls_por_marca["Semana"] == 14]
resumen_rqls_por_marca_week["brand"].unique()

array(['Bajaj', 'Dinamo', 'Honda', 'Vento', 'Yamaha', 'Suzuki', 'Hero',
       'Italika', 'TVS', 'Zmoto', 'Kawasaki', 'CF Moto', 'CFLITE', 'KTM',
       'MB', 'Benelli', 'Treck', 'Kymco', 'Royal Enfield', 'Husqvarna',
       'Jiajue', 'Zontes'], dtype=object)

In [14]:
resumen_rqls_por_marca_week_copy = resumen_rqls_por_marca_week.copy()

test = resumen_rqls_por_marca_week_copy.drop(columns=["state", "day"])

In [15]:
test_sum_rql_per_brand =test.groupby(["brand", "Semana", "Año"])["rql"].sum().reset_index()
test_sum_rql_per_brand

,brand,Semana,Año,rql
0,Bajaj,14,2025,1592.0
1,Benelli,14,2025,66.0
2,CF Moto,14,2025,433.0
3,CFLITE,14,2025,106.0
4,Dinamo,14,2025,23.0
5,Hero,14,2025,45.0
6,Honda,14,2025,291.0
7,Husqvarna,14,2025,6.0
8,Italika,14,2025,283.0
9,Jiajue,14,2025,3.0


In [16]:
test_sum_rql_per_brand["rql"].sum()
print(f"Total de rql: {test_sum_rql_per_brand['rql'].sum()}")

Total de rql: 4027.0
